In [1]:
import os as os

import numpy as np

import tensorflow.keras as keras
import tensorflow.train as train
import tensorflow.nn as nn

from sklearn.model_selection import train_test_split

In [2]:
def convertLabel(sequence): #gives a nb to the aminoacid
    label = sequence[7]
    aminoacidList = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE",
                    "LEU", "LYS", "MET", "PHE", "PRO", "PYL", "SEL", "SER", "THR", "TRP", "TYR", "VAL"]
    for i in range (0, 22):
        if label == aminoacidList[i]:
            return i

In [3]:
def load_data():


    labelList = []

    with open('matrix.csv') as f:

        nbOfSamples = int(f.readline())
        i = 0

        shapes = f.readline() # length of sequence + nb of features
        input_shape = int(shapes.split(",")[1])


        distancesList = np.zeros((nbOfSamples, input_shape))
        labelList = np.zeros(nbOfSamples)


        while i < nbOfSamples:
            # Extracting labels
            sequence = f.readline().split(",")
            labelList[i] = convertLabel(sequence)

            # Extracting distances
            distancesString = f.readline()
            distancesArray = np.array([float(s) for s in distancesString.split(",")])
            distancesList[i, :] = distancesArray

            # empty line
            f.readline()

            i += 1
    return input_shape, distancesList, labelList

In [4]:
# returns a short sequential model
def create_model():
    # activation function: leaky ReLU
    leakyrelu = lambda x: keras.activations.relu(x, alpha=0.01, max_value=None, threshold=0)

    # creation du reseau de neurones
    model = keras.models.Sequential([

        # hidden layer
        keras.layers.Dense(units = 500, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 300, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 200, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 100, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 75, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 53, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),

        # final layer
        keras.layers.Dense(units = 22, activation = nn.softmax),
    ])

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

In [5]:
# Data processing
input_shape, distances, labels = load_data()


x_train, x_test, y_train, y_test = train_test_split(
    distances, labels, test_size=0.2, random_state=42)

In [6]:
# Create checkppoints

checkpoint_path = "checkpoints/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weights_only=True, verbose=1,
    # save weights, every 5-epoch
    period=5)

latest = train.latest_checkpoint(checkpoint_dir)

In [7]:
# create a model instance

model = create_model()

if latest: model.load_weights(latest)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# training
model.fit(x_train, y_train, epochs = 500,
          validation_data = (x_test, y_test),
          callbacks = [cp_callback]) # save checkpoints at the end of each epoch

model.save('my_model.h5')